In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
import scipy.sparse as sp
import torch.nn as nn
from node2vec import Node2Vec
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import itertools
import dgl

In [36]:
from dgl.nn import SAGEConv
from dgl.nn import GraphConv
import dgl.function as fn

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

class GraphGCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphGCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, allow_zero_in_degree=True)
        self.conv2 = GraphConv(h_feats, h_feats, allow_zero_in_degree=True)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [37]:
G_5day = nx.read_gexf("../graph_creation/2020-07-01__to__2020-07-05__15-overlap__hashmap.gexf")
G_6day = nx.read_gexf("../graph_creation/2020-07-01__to__2020-07-06__15-overlap__hashmap.gexf")

In [128]:
sorted(G_5day.nodes())

['...',
 ...]

In [130]:
node_mapping = {i: x for i, x in enumerate(sorted(G_5day.nodes()))}

In [131]:
node_mapping[0]

'000433eca9a8548de1efa8902b3b3b5d0ee9fa2acf653a265cc086aaaff08c94'

In [132]:
count = 0
for edge in G_5day.edges():
    if (edge[0] == node_mapping[1]) or (edge[1] == node_mapping[1]):
        count += 1
print(count)

84


In [133]:
dgl_G_5day = dgl.from_networkx(G_5day)
dgl_G_6day = dgl.from_networkx(G_6day)

In [134]:
day5_edges = dgl_G_5day.edges()
day6_edges = dgl_G_6day.edges()
count_2 = 0
for i in range(len(day5_edges[0])):
    if (day5_edges[0][i] == 0) or (day5_edges[1][i] == 0):
        count_2 += 1
print(count_2)

168


In [137]:
len(day6_edges[0])

2003742

In [138]:
len(G_6day.edges())

1001871